In [3]:
import pandas as pd
from datetime import datetime
import numpy as np
from datetime import timedelta, date

import matplotlib.dates as mdates
from scipy import stats
import datetime as dt


In [4]:
import psycopg2
connection = psycopg2.connect(user = "postgres", password = "admin", host = "127.0.0.1",port = "5432", database = "postgres")

In [123]:
def process_data(quotes):
    from talib import RSI
    from talib import MACD
    from talib import SMA
    from talib import EMA
    from talib import ATR
    # Simple Moving Averages
    quotes['sma50'] = SMA(quotes['close'], timeperiod=50)
    # Exponential Moving Averages
    quotes['ema50'] = EMA(quotes['close'], timeperiod=50)
    # MACD indicator - MACD(12,26,9)
    quotes['macd_line'], quotes['signal_line'], quotes['macd_hist'] = MACD(quotes['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    # Compute RSI (14 days)
    quotes["rsi"] = RSI(quotes['close'].values, timeperiod=14)
    quotes["atr"] = ATR(quotes['high'].values,quotes['low'].values, quotes['close'].values ,timeperiod=14)
    
    atr_stoploss  = ATR_STOPLOSS(close = quotes.close.values, high = quotes.high.values, low = quotes.low.values, atr = quotes.atr.values, times=5, period=14)
    #quotes = quotes.append(pd.Series(), ignore_index=True)
    quotes["atr_stoploss"] = atr_stoploss[0]
    return quotes


def ATR_STOPLOSS(close, high, low, atr, times=4, period=14):
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=close[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=close[i]-loss  
                continue
                
        if(math.isnan(atr[i])==False):
            loss = atr[i]*times
            atr_trailing_stoploss[i+1]=close[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>close[i]+loss):
                    atr_trailing_stoploss[i+1]=close[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<close[i]-loss):
                    atr_trailing_stoploss[i+1]=close[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]

In [127]:
def close_conn():
    if(connection):
        cursor.close()
        connection.close()
        
stock = 'AAPL'
data = pd.read_sql_query("SELECT * FROM finance.stock_price where stock = '"+stock+"' and date > '2010-01-01' order by date asc",con=connection)
processed_df= process_data(data)

In [129]:
processed_df

,stock,open,high,low,close,adj_close,vol,date,sma50,ema50,macd_line,signal_line,macd_hist,rsi,atr,atr_stoploss
0,AAPL,30.490000,30.642857,30.340000,30.572857,26.601469,123432400,2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,30.657143,30.798570,30.464285,30.625713,26.647457,150476200,2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL,30.625713,30.747143,30.107143,30.138570,26.223597,138040000,2010-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,30.250000,30.285715,29.864286,30.082857,26.175120,119282800,2010-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAPL,30.042856,30.285715,29.865715,30.282858,26.349140,111902700,2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,AAPL,258.300000,262.470000,258.280000,262.200000,262.200000,20455300,2019-11-11,231.6190,234.901017,8.711304,7.917839,0.793465,79.690856,4.139224,239.463412
2482,AAPL,261.550000,262.790000,260.920000,261.960000,261.960000,21847200,2019-11-12,232.7442,235.962153,8.714928,8.077257,0.637672,79.051157,3.977136,241.503882
2483,AAPL,261.130000,264.780000,261.070000,264.470000,264.470000,25683600,2019-11-13,233.8498,237.080108,8.818680,8.225541,0.593139,80.788096,3.958055,242.074319
2484,AAPL,263.750000,264.880000,262.100000,262.640000,262.640000,22295700,2019-11-14,234.8370,238.082457,8.653487,8.311131,0.342356,75.850184,3.873908,244.679725


###Plot chart

In [130]:
######Plot######
import plotly.graph_objs as go 
from datetime import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.subplots import make_subplots
# Make sure dates are in ascending order
# We need this for slicing in the callback below
df = processed_df.set_index('date')

fig = go.FigureWidget(make_subplots(rows=4, cols=1))
fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['close']),
    name = "close"),
    row=1, col=1
)
fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['ema50']),
    name = "ema50"),
    row=1, col=1
)
fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['atr_stoploss']),
    name = "atr_stoploss"),
    row=1, col=1
)

fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['vol']),
    name = "vol"),
    row=2, col=1
)
fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['rsi']),
    name = "rsi"),
    row=3, col=1
)

fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['macd_line']),
    name = "macd_line"),
    row=4, col=1
)
fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['signal_line']),
    name = "signal_line"),
    row=4, col=1
)

fig.add_trace(
    go.Scattergl(x=list(df.index), y=list(df['macd_hist']),
    name = "macd_hist"),
    row=4, col=1
)


fig.layout.xaxis=dict(
        anchor='x',
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='YTD',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        type='date'
    )
fig.layout.yaxis=dict(
        domain=[0.6, 1]
    )
fig.layout.yaxis2=dict(
        domain=[0.46, 0.57]
    )
fig.layout.yaxis3=dict(
        domain=[0.23, 0.43]
    )
fig.layout.yaxis4=dict(
        domain=[0, 0.2]
    )
fig.layout.yaxis3.range=[0,100]
fig.layout.yaxis3.tickvals=[0,20,80,100]
fig['layout'].update(height=800, width=1000) 
fig['layout'].update(title=stock) 

    
#trace = go.Candlestick(x=df.index,open=df['open'],high=df['high'],low=df['low'],close=df['close'])

import time
import datetime
def zoom(layout, xrange):
    try:
        in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
        fig.layout.xaxis2.range = fig.layout.xaxis.range
        fig.layout.xaxis3.range = fig.layout.xaxis.range
        fig.layout.xaxis4.range = fig.layout.xaxis.range
        padding = 0.1
        
        from_price = in_view.iloc[0].close
        to_price = in_view.iloc[-1].close
        growth = (to_price-from_price)/from_price*100.0
        max_val_y = in_view.close.max()
        min_val_y =  in_view.close.min()
        axis_diff = max_val_y-min_val_y
        fig.layout.yaxis.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
        
        fig.layout.yaxis2.range = [0, in_view.vol.max()]
     
        fig['layout'].update(title=stock+" ("+str(round(growth,2))+"%)") 
        max_val_y1 = max(abs(in_view.macd_line.max()),abs(in_view.macd_line.min()))
        max_val_y2 = max(abs(in_view.signal_line.max()),abs(in_view.signal_line.min()))
        max_val_y=max(max_val_y1,max_val_y2)
        min_val_y = -max_val_y
        axis_diff = max_val_y-min_val_y
        fig.layout.yaxis4.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
    except Exception as e:
        None
fig.layout.on_change(zoom, 'xaxis')
fig

FigureWidget({
    'data': [{'name': 'close',
              'type': 'scattergl',
              'uid': '744ffa1…